```{=latex}
\usepackage{hyperref}
\usepackage{graphicx}
\usepackage{listings}
\usepackage{textcomp}
\usepackage{fancyvrb}

\newcommand{\passthrough}[1]{\lstset{mathescape=false}#1\lstset{mathescape=true}}
```

```{=latex}
\title{Caching Best Practices}
\author{Moshe Zadka -- https://cobordism.com}
\date{2021}

\begin{document}
\begin{titlepage}
\maketitle
\end{titlepage}

\frame{\titlepage}
```

```{=latex}
\begin{frame}
\frametitle{Acknowledgement of Country}

Belmont (in San Francisco Bay Area Peninsula)

Ancestral homeland of the Ramaytush Ohlone

\end{frame}
```

## How caching works (15m)

### Caching (5m)

#### What is a cache

```{=latex}
\begin{frame}
\frametitle{What is a Cache}

(For the purposes of this talk)

\begin{itemize}
\item Key/value
\item In-memory
\item Not pesistent
\end{itemize}
\end{frame}
```

#### What is not a cache

```{=latex}
\begin{frame}
\frametitle{Not a Cache}

\begin{itemize}
\item Reliable
\item Communication
\end{itemize}
\end{frame}
```

#### Caching for latency

```{=latex}
\begin{frame}
\frametitle{Why Cache? Latency}

Long computations
\end{frame}
```

#### Caching for resource

```{=latex}
\begin{frame}
\frametitle{Why Cache? Resources}

Expensive computations
\end{frame}
```

#### Caching -- different objectives

```{=latex}
\begin{frame}
\frametitle{Why Cache? Examples}

\begin{itemize}
\item Latency: Username retrieved from different datacenter
\item Resources: Indexed DB join
\end{itemize}

\end{frame}
```

### Memcache overview (5m)

#### Memcache

```{=latex}
\begin{frame}
\frametitle{Memcache}

\begin{itemize}
\item Popular cache
\item Fast
\item Does one thing
\item Focus of this talk
\end{itemize}

\end{frame}
```

#### Semantics

```{=latex}
\begin{frame}[fragile]
\frametitle{Memcache interface}
```

In [1]:
# Simplified from pymemcahce API
class MemcacheClient:
    def set(self, key, value):
        ...
    def get(self, key):
        ...
    def get_many(self, keys):
        ...
    def delete(self, key):
        ...

```{=latex}
\end{frame}
```

#### Behavior

```{=latex}
\begin{frame}
\frametitle{Memcache key lifecycle}

\begin{itemize}
\item set
\item get
\item expire/delete/expeunge
\end{itemize}

\end{frame}
```

### Cache routing (5m)

#### What is Routing?

```{=latex}
\begin{frame}
\frametitle{Memcache routing}

(Like HTTP proxies) \pause

Examples: \pause

mcrouter \pause

twemproxy
\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Memcache client-side routing}

Client responsible for: \pause

\begin{itemize}
\item Detecting server outage and fail-over
\item "Fair" sharding of keys
\end{itemize}

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Memcache: Routing is necessary}

Redundancy

Scaling
\end{frame}
```

### Caching basics

```{=latex}
\begin{frame}
\frametitle{Caching points}

\begin{itemize}
\item Use: optimize latency or resources
\item Keys can vanish
\item Routing necessary (client or server)
\end{itemize}

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Caching tier example}

\begin{itemize}
\item McRouter layer
\item Memcache layer
\item TCP/DNS load-balancing for McRouter
\end{itemize}

\end{frame}
```

## Correctness (15m)

```{=latex}
\begin{frame}
\frametitle{Caching complicates code}

Potentially introduces new bugs

\end{frame}
```

### Caching errors (5m)

```{=latex}
\begin{frame}
\frametitle{Caching adds corner cases}

Code needs to account for them.

\end{frame}
```

#### Disconnections

```{=latex}
\begin{frame}
\frametitle{Network: Disconnection}

Persistent connection disconnects

\end{frame}
```

#### Timeouts

```{=latex}
\begin{frame}
\frametitle{Network: Timeouts}

Packets go missing

Servers go offline

\end{frame}
```

#### Missing keys

```{=latex}
\begin{frame}
\frametitle{Keys: Missing}

Server rebooted

Expiry

...and more

\end{frame}
```

#### Invalid values

```{=latex}
\begin{frame}
\frametitle{Values: Invalid}

Key collision

Problem in previous version

\end{frame}
```

### Cache invalidation (5m)

```{=latex}
\begin{frame}
\frametitle{Values: Outdated}

Things change

\end{frame}
```

### Hard!

```{=latex}
\begin{frame}
\frametitle{Cache invalidation}

Hard

\end{frame}
```

#### Expiring keys

```{=latex}
\begin{frame}
\frametitle{Cache expiration}

Upper limit on staleness

\end{frame}
```

#### Invalidating keys

```{=latex}
\begin{frame}
\frametitle{Cache delete}

Mixed blessing

\end{frame}
```

### Testing (5m)

```{=latex}
\begin{frame}
\frametitle{Cache adds complexity}

More ways to fail...\pause

especially under pressure.
\end{frame}
```

#### Test cache hit

```{=latex}
\begin{frame}
\frametitle{Test cache hit path}

Often missed in unit tests

\end{frame}
```

#### Test cache miss

```{=latex}
\begin{frame}
\frametitle{Test cache miss path}

...in realistic scenarios.

\end{frame}
```

#### Test "garbage" cache value

```{=latex}
\begin{frame}
\frametitle{Test bad cache value}

If you update your software,
can you detect an old cache value?

\end{frame}
```

#### Test network disconnections

```{=latex}
\begin{frame}
\frametitle{Test cache networking: disconnection}

Will happen at the weirdest times.

\end{frame}
```

#### Test network timeouts

```{=latex}
\begin{frame}
\frametitle{Test cache networking: timeouts}

How much can the cache slow you down?

\end{frame}
```

## Performance (15m)

```{=latex}
\begin{frame}
\frametitle{Cache is an optimization}

Treat it as one

\end{frame}
```

#### Measure (5m)

```{=latex}
\begin{frame}
\frametitle{Measure cache effectiveness}

Is it helpful? How much?

\end{frame}
```

#### Measure ratio per type

```{=latex}
\begin{frame}
\frametitle{Measure ratio by code path}

Each code path/object type should have its own counter

Counters are good

\end{frame}
```

#### Distinct timers for hit/miss

```{=latex}
\begin{frame}
\frametitle{Measure timer histogram by hit/miss}

Is there a difference? Enough?

\end{frame}
```

#### Allow on/off

```{=latex}
\begin{frame}
\frametitle{Configuration: Dummy cache}

Cache object that always "miss"

\begin{itemize}
\item Debugging
\item A/B performance
\item Remediation
\end{itemize}

\end{frame}
```

#### Measure A/B performance

```{=latex}
\begin{frame}
\frametitle{Performance: A/B}

By key hash

\end{frame}
```

### What to cache (5m)

```{=latex}
\begin{frame}
\frametitle{What to cache?}

Granularity?\pause

Think + Test
\end{frame}
```

#### Size

```{=latex}
\begin{frame}
\frametitle{Cache: Size concerns}

Caching can cause eviction

\end{frame}
```

#### Serialization

```{=latex}
\begin{frame}
\frametitle{Cache: Serialization}


JSON?\pause
Pickle?

\end{frame}
```

### Other performance (5m)

#### Socket options

```{=latex}
\begin{frame}[fragile]
\frametitle{Cache: Socket options}

\verb|TCP_NODELAY|?\pause

\verb|TCP_QUICKACK|?\pause

\verb|TCP_CORK|?

\end{frame}
```

#### Multi-get

```{=latex}
\begin{frame}
\frametitle{Cache: Multiget}

Refactoring might be required

\end{frame}
```

## Take-aways (5m)

```{=latex}
\begin{frame}
\frametitle{Takeaways}

\begin{itemize}
\item Caching is an optimization
\item Test correctness
\item Monitor performance
\item Apply best practices
\end{itemize}

\end{frame}
```

```{=latex}
\end{document}
```